# Влияние близости школ: регрессия и парные сравнения

Цель: более строго проверить гипотезу, что у магазинов рядом со школами
- **выше пешеходный трафик**,
- **ниже средний чек** (как минимум в части сегментов).

В этом ноутбуке:
1. Готовим признаки и фильтруем данные.
2. Строим две линейные регрессии (трафик и чек).
3. Делаем парные сравнения на уровне (НП, размер магазина).


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

input_path = Path('Х5_with_region_index_2024_population_patched_with_flags.xlsx')
df = pd.read_excel(input_path)
print('Размер исходного датафрейма:', df.shape)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Х5_with_region_index_2024_population_patched_with_flags.xlsx'

## Подготовка признаков

Фильтруем только нормальные наблюдения и заводим нужные признаки: has_school, size_group, population, ria_index.

In [3]:
col_np = 'Населенный пункт'
col_region = 'Регион'
col_index = 'Индекс_РИА_2024'
col_is_season = 'is_season'
col_traffic_flag = 'traffic_flag'
col_pedestrian = 'Трафик пеший, в час'
col_check = 'Средний чек'
col_schools = 'Школы (300 м)'
col_size_cat = 'Торговая площадь, категориальный'

df[col_index] = pd.to_numeric(df[col_index], errors='coerce')
df[col_pedestrian] = pd.to_numeric(df[col_pedestrian], errors='coerce')
df[col_check] = pd.to_numeric(df[col_check], errors='coerce')
df[col_schools] = pd.to_numeric(df[col_schools], errors='coerce').fillna(0)

df['has_school'] = df[col_schools] > 0

size_map = {
    'Маленький': 'small',
    'Средний': 'medium',
    'Большой': 'large_plus',
    'Очень большой': 'large_plus',
}
df['size_group'] = df[col_size_cat].map(size_map)

# ищем колонку с численностью населения
pop_col = None
for c in df.columns:
    name = str(c)
    if 'числен' in name.lower() and 'насел' in name.lower():
        pop_col = c
        break
print('Колонка с населением:', pop_col)

if pop_col is not None:
    df[pop_col] = pd.to_numeric(df[pop_col], errors='coerce')
    df['population'] = df[pop_col]
else:
    df['population'] = np.nan

mask = (
    (df[col_is_season] == 1)
    & (df[col_traffic_flag] == 1)
    & (df[col_pedestrian] > 0)
    & df[col_index].notna()
    & df['size_group'].notna()
)

base_df = df[mask].copy()
print('Размер base_df после фильтрации:', base_df.shape)
base_df[[col_np, col_region, col_pedestrian, col_check, col_index, 'population', 'has_school', 'size_group']].head()

Колонка с населением: Численность населения
Размер base_df после фильтрации: (60395, 24)


,Населенный пункт,Регион,"Трафик пеший, в час",Средний чек,Индекс_РИА_2024,population,has_school,size_group
3,"1-го отделения совхоза ""Масловский"" п",Воронежская обл,89.857143,1119.028697,61.98,3827,False,medium
4,"1-го отделения совхоза ""Масловский"" п",Воронежская обл,89.857143,1112.584778,61.98,3827,False,medium
10,"1-го отделения совхоза ""Масловский"" п",Воронежская обл,89.857143,1275.068118,61.98,3827,False,medium
13,1-я Моква д,Курская обл,95.666667,683.123700,50.02,500,True,medium
15,1-я Моква д,Курская обл,95.666667,687.641711,50.02,500,True,medium


## Подготовка данных для регрессии

Переименуем ключевые переменные и добавим логарифмы.

In [4]:
reg_df = base_df.copy()
reg_df = reg_df.rename(columns={
    col_np: 'np_name',
    col_region: 'region',
    col_pedestrian: 'traffic_ped',
    col_check: 'avg_check',
    col_index: 'ria_index',
})

# убираем строки без населения, если оно есть
if reg_df['population'].notna().any():
    reg_df = reg_df[reg_df['population'].notna()].copy()

reg_df = reg_df[(reg_df['traffic_ped'] > 0) & (reg_df['avg_check'] > 0)].copy()
reg_df['log_traffic'] = np.log(reg_df['traffic_ped'])
reg_df['log_check'] = np.log(reg_df['avg_check'])
reg_df['log_population'] = np.log(reg_df['population'].replace({0: np.nan}))

print('Размер reg_df для регрессии:', reg_df.shape)
reg_df[['np_name', 'region', 'traffic_ped', 'avg_check', 'ria_index', 'population', 'has_school', 'size_group']].head()

Размер reg_df для регрессии: (60392, 27)


,np_name,region,traffic_ped,avg_check,ria_index,population,has_school,size_group
3,"1-го отделения совхоза ""Масловский"" п",Воронежская обл,89.857143,1119.028697,61.98,3827,False,medium
4,"1-го отделения совхоза ""Масловский"" п",Воронежская обл,89.857143,1112.584778,61.98,3827,False,medium
10,"1-го отделения совхоза ""Масловский"" п",Воронежская обл,89.857143,1275.068118,61.98,3827,False,medium
13,1-я Моква д,Курская обл,95.666667,683.123700,50.02,500,True,medium
15,1-я Моква д,Курская обл,95.666667,687.641711,50.02,500,True,medium


## Линейная регрессия: трафик

In [5]:
import statsmodels.formula.api as smf

traffic_model = smf.ols(
    formula='log_traffic ~ has_school + C(size_group) + log_population + ria_index + C(region)',
    data=reg_df,
).fit()

print(traffic_model.summary().tables[1])
print('\nКоэффициент has_school по трафику (лог):', traffic_model.params.get('has_school[T.True]', np.nan))

                                                               coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------------
Intercept                                                    3.5933      0.064     56.548      0.000       3.469       3.718
has_school[T.True]                                           0.0959      0.005     20.560      0.000       0.087       0.105
C(size_group)[T.medium]                                     -0.1144      0.005    -21.541      0.000      -0.125      -0.104
C(size_group)[T.small]                                      -0.1168      0.006    -18.209      0.000      -0.129      -0.104
C(region)[T.Алтай Респ]                                      1.0392      0.092     11.256      0.000       0.858       1.220
C(region)[T.Алтайский край]                                  0.5412      0.032     16.775      0.000       0.478       0.604


Если коэффициент `has_school` положительный и значимый (p-value < 0.05),
это означает: при равных индексе, населении, размере магазина и регионе
магазины рядом со школой имеют более высокий пешеходный трафик.

## Линейная регрессия: средний чек

In [6]:
check_model = smf.ols(
    formula='log_check ~ has_school + C(size_group) + log_population + ria_index + C(region)',
    data=reg_df,
).fit()

print(check_model.summary().tables[1])
print('\nКоэффициент has_school по чеку (лог):', check_model.params.get('has_school[T.True]', np.nan))

                                                               coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------------
Intercept                                                    6.7758      0.031    217.296      0.000       6.715       6.837
has_school[T.True]                                          -0.0294      0.002    -12.865      0.000      -0.034      -0.025
C(size_group)[T.medium]                                     -0.1998      0.003    -76.644      0.000      -0.205      -0.195
C(size_group)[T.small]                                      -0.3414      0.003   -108.483      0.000      -0.348      -0.335
C(region)[T.Алтай Респ]                                      0.3491      0.045      7.707      0.000       0.260       0.438
C(region)[T.Алтайский край]                                 -0.1485      0.016     -9.379      0.000      -0.180      -0.117


Отрицательный и значимый коэффициент `has_school` по `log_check`
будет означать, что средний чек у магазинов рядом со школой ниже,
чем у сопоставимых магазинов без школы (при контроле остальных факторов).

## Парные сравнения по (НП, size_group)

Теперь сделаем более "интуитивную" проверку:
- На уровне каждой пары `(Населенный пункт, size_group)` сравниваем средний трафик/чек
  среди магазинов со школой и без школы.
- Строим t-тест по разностям.

In [ ]:
from scipy import stats

group_cols = [col_np, 'size_group']

agg = (
    base_df
    .groupby(group_cols + ['has_school'])
    .agg(
        mean_traffic=(col_pedestrian, 'mean'),
        mean_check=(col_check, 'mean'),
        count=('has_school', 'size'),
    )
    .reset_index()
)

pivot_pairs = agg.pivot_table(
    index=group_cols,
    columns='has_school',
    values=['mean_traffic', 'mean_check'],
)

pivot_pairs.columns = [f"{metric}_{'with' if hs else 'without'}" for metric, hs in pivot_pairs.columns]
pivot_pairs = pivot_pairs.dropna().reset_index()

pivot_pairs['diff_traffic'] = pivot_pairs['mean_traffic_with'] - pivot_pairs['mean_traffic_without']
pivot_pairs['diff_check'] = pivot_pairs['mean_check_with'] - pivot_pairs['mean_check_without']

print('Примеры пар:')
pivot_pairs.head()

### t-тесты по разностям

Отдельно для small / medium / large_plus считаем t-тесты по разностям
`with_school - without_school` для трафика и чека.

In [ ]:
alpha = 0.05

for size in ['small', 'medium', 'large_plus']:
    sub = pivot_pairs[pivot_pairs['size_group'] == size]
    if sub.empty:
        print(f"\n[WARN] Нет пар для size_group={size}")
        continue
    dif_tr = sub['diff_traffic'].dropna()
    dif_ch = sub['diff_check'].dropna()
    print(f"\n=== size_group = {size} ===")
    print('Число пар (НП):', len(sub))

    if len(dif_tr) >= 3:
        t_tr, p_tr = stats.ttest_1samp(dif_tr, popmean=0.0)
        print(f"Трафик: mean_diff={dif_tr.mean():.2f}, t={t_tr:.3f}, p={p_tr:.4g}")
        if p_tr < alpha:
            print('→ Разница по трафику статзначима (в среднем у школ трафик иной, чем без школ)')
    else:
        print('Недостаточно пар для трафика')

    if len(dif_ch) >= 3:
        t_ch, p_ch = stats.ttest_1samp(dif_ch, popmean=0.0)
        print(f"Чек: mean_diff={dif_ch.mean():.2f}, t={t_ch:.3f}, p={p_ch:.4g}")
        if p_ch < alpha:
            print('→ Разница по чеку статзначима (в среднем у школ чек иной, чем без школ)')
    else:
        print('Недостаточно пар для чека')